In [1]:
import pandas as pd  # 导入pandas库，用于数据处理和分析
import numpy as np  # 导入numpy库，用于科学计算和数值计算
import sklearn  # 导入scikit-learn库，用于机器学习和数据挖掘
import lightgbm as lgb  # 导入lightgbm库，用于梯度提升决策树算法
from sklearn.preprocessing import OrdinalEncoder  # 导入OrdinalEncoder类，OrdinalEncoder类用于将类别型特征编码成数值型特征，以便用于机器学习算法中

In [2]:
# 导入train.csv文件的数据，将其存储在Xy_train变量中
Xy_train = pd.read_csv('train.csv')

# 导入test.csv文件的数据，将其存储在X_test变量中
X_test = pd.read_csv('test.csv')

In [3]:
# 按行将Xy_train和X_test拼接起来
Xy_all = pd.concat([Xy_train, X_test])

# 获取所有分类变量的特征名称
cat_features = Xy_all.columns[Xy_all.dtypes==object]


In [4]:
# 定义一个OrdinalEncoder对象oridinalEncoder，用于对分类特征进行编码
# 设置编码后的数据类型为np.int32
# 当遇到未知的特征时，使用编码值-1来表示
# 缺失值也被编码为-1
oridinalEncoder = OrdinalEncoder(
    dtype=np.int32,
    handle_unknown='use_encoded_value',
    unknown_value=-1,
    encoded_missing_value=-1
).set_output(transform='pandas')

# 使用oridinalEncoder对cat_features所包含的分类特征进行编码，并将编码结果赋值给Xy_all中的对应列
Xy_all[cat_features] = oridinalEncoder.fit_transform(Xy_all[cat_features])


In [5]:
# 首先，使用isna()函数检查'SalePrice'是否为NaN
has_sale_price = ~Xy_all['SalePrice'].isna()
no_sale_price = Xy_all['SalePrice'].isna()

# 接下来，使用query()函数将非NaN的行和NaN的行分别筛选出来
Xy_all_with_sale_price = Xy_all[has_sale_price]
Xy_all_without_sale_price = Xy_all[no_sale_price]

# 将原始数据集中除去'SalePrice'列后的所有数据作为训练集的特征数据
X_train = Xy_all_with_sale_price.drop(['SalePrice'], axis=1)

# 将原始数据集中的'SalePrice'列作为训练集的标签数据
y_train = Xy_all_with_sale_price['SalePrice']

# 将原始数据集中除去'SalePrice'列后的所有数据作为测试集的特征数据
X_test = Xy_all_without_sale_price.drop(['SalePrice'], axis=1)

In [6]:
# 创建一个lgb.Dataset对象，用于训练模型
dtrain = lgb.Dataset(X_train, label=y_train)

# 定义模型的参数
params = {
    'boosting_type': 'gbdt',  # 指定为 gbdt 类型的增强方法，如 gradient boosting decision tree。
    'num_leaves': 31,  # 指定叶节点数。
    'max_depth': -1,  # 允许树的最大深度为无穷大。
    'learning_rate': 0.1,  # 决策树在梯度下降法中的步长。
    'n_estimators': 100,  # 随机森林中决策树的数量。
    'subsample_for_bin': 200000,  # 用于创建二进制索引的样本数目。
    'objective': None,  # 模型的目标函数，此处未指定，需根据模型类型确定。
    'class_weight': None,  # 对于不平衡数据集，可以指定权重。
    'min_split_gain': 0.0,  # 分裂节点时最小的信息增益阈值。
    'min_child_weight': 0.001,  # 每个叶节点中最小权重的样本权重和。
    'min_child_samples': 20,  # 每个叶节点中最小的样本数。
    'subsample': 1.0,  # 随机特征子集的样本比例。
    'subsample_freq': 0,  # 多少次迭代使用一次特征子集。
    'colsample_bytree': 1.0,  # 随机特征行的样本比例。
    'reg_alpha': 0.0,  # L1正则化参数，也称为AlphaL1。
    'reg_lambda': 0.0,  # L2正则化参数，也称为LambdaL2。
    'random_state': None,  # 随机种子，用于设置随机数生成器的随机起点。
    'n_jobs': None  # 使用的并行线程数。
}

# 可能需要根据实际模型类型调整 importance_type 参数。
# 'split' 指的是基于节点分裂的特征重要性度量，
# 'weight' 指的是基于节点样本权重的特征重要性度量。
# 'importance_type': 'split'  # 特征重要性的计算方式。

# 创建lightgbm模型并训练
model = lgb.train(params, dtrain, num_boost_round=100)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003282 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3646
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 75
[LightGBM] [Info] Start training from score 180921.195890


In [7]:
X_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,2,80.0,11622,1,-1,3,3,0,...,120,0,-1,2,-1,0,6,2010,8,4
1,1462,20,3,81.0,14267,1,-1,0,3,0,...,0,0,-1,-1,0,12500,6,2010,8,4
2,1463,60,3,74.0,13830,1,-1,0,3,0,...,0,0,-1,2,-1,0,3,2010,8,4
3,1464,60,3,78.0,9978,1,-1,0,3,0,...,0,0,-1,-1,-1,0,6,2010,8,4
4,1465,120,3,43.0,5005,1,-1,0,1,0,...,144,0,-1,-1,-1,0,1,2010,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,4,21.0,1936,1,-1,3,3,0,...,0,0,-1,-1,-1,0,6,2006,8,4
1455,2916,160,4,21.0,1894,1,-1,3,3,0,...,0,0,-1,-1,-1,0,4,2006,8,0
1456,2917,20,3,160.0,20000,1,-1,3,3,0,...,0,0,-1,-1,-1,0,9,2006,8,0
1457,2918,85,3,62.0,10441,1,-1,3,3,0,...,0,0,-1,2,2,700,7,2006,8,4


In [8]:
# 使用训练好的模型对测试集进行预测
y_pred = model.predict(X_test)

# 创建一个DataFrame，包含测试集的索引和预测结果
submission = pd.DataFrame({'Id': X_test.Id, 'SalePrice': y_pred})

# 输出到csv文件
submission.to_csv('first.csv', index=False)